# Scrape Mars Data: The News

In [1]:
# Import Splinter and BeautifulSoup
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager

import pandas as pd

In [2]:
# Set up executable path
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======


ValueError: Could not get version for Chrome with this command: reg query "HKEY_CURRENT_USER\Software\Google\Chrome\BLBeacon" /v version

### Assign the url and instruct the browser to visit it

In [3]:
# Visit the mars nasa news site
url = 'https://redplanetscience.com'
browser.visit(url)
# Optional delay for loading the page
browser.is_element_present_by_css('div.list_text', wait_time=1)
    # NOTE: This line of code accomplishes two things:
    # 1. Searchng for elements with specific combo of tag (div) and attribute (list_text)
        # ex. `ul.item_list` -> HTML as `<ul class="item_list">`
    # 2. Telling browser to wait one second before searching for components

NameError: name 'browser' is not defined

In [ ]:
# Set up HTML parser
html = browser.html
news_soup = soup(html, 'html.parser')
slide_elm = news_soup.select_one('div.list_text')
    # NOTE: CSS works from right to left: returnig last item on list instead of first
        # select_one : first matching element returned will be a <li /> element w/ class=slide + all nested elements within it

In [ ]:
# Q: Which HTMLL attribute will we use to scrape the article's title?
# A: class = "content_title" 

In [ ]:
# Assing the title and summary text to variables to begin scrapping
slide_elem.find('div', class_=content_title)

In [ ]:
# Use the parent element to find the first 'a' tag and save it as `news_title`
news_title = slide_elem.find('div', class_='content_title').get_text()
news_title

### Next add the summary text : Change class to "article_teaser_body"

In [ ]:
# Use the parent element to find the paragraph text
news_p = slide_elem.find('div', class_= 'article_teaser_body').get_text()
news_p

# Scrape Mars Data: Featured Image

In [ ]:
# Visit URL
url = 'https://spaceimages-mars.com'
browser.visit(url)

In [ ]:
# Find and click the full image button
full_image_elem = browser.find_by_tag('button')[1] # Index : Want broswer to click second button
full_image_elem.click()

In [ ]:
# Parse the resulting html with soup
html = browser.html
img_soup = soup(html, 'html.parser')

In [ ]:
# Find the relative image url
img_url_rel = img_soup.find('img', class_='fancybox-image').get('src')

# BREAK DOWN CODE:
    # 1. An `img` tag is nested within this HTML, so we've included it
    # 2. `.get('src') pulls the link to the image
    
# Code allows us to pull link to image by pointing BeautifulSoup to where image will be, instead of grabbing URL directly.
# When JPL updates image page, our code will still pull the most recent image

In [ ]:
# Use the base URL to create an absolute URL
img_url = f'https://spaceimages-mar.com/{img_url_rel}' # f-string evaluted at run-time - print statement and variable it holds will be updated frquently as data we are scrapping is live.
img_url


# Scrape Mars Data: Mars Facts

In [ ]:
# Scrape the entire table with Pandas' .read_html() funciton
df = pd.read_html('https://galaxyfacts-mars.com')[0]
df.columns = ['description', 'Mars', 'Earth']
df.set_index('description', inplace=True)
df

In [ ]:
# Convert dataframe back into HTML-ready code
df.to_html()

In [ ]:
# End automated browsing session
browser.quit()